# IBM-CHCP-AIMS Workshop on The Theory of Quantum Learning Algorithms
## Quantum Machine Learning Practical Session with Qiskit

## Table of Contents

0. [Introduction to the QML Practical Session with Qiskit!](#welcome)<br>
0.1. [Verifying Qiskit installation and checking version](#install)<br>
0.2. [Import Modules and Download IRIS Dataset](#import)<br>
0.3. [Overview of the QML Pipeline Adapted from Qiskit Patterns](#introduction)<br>
2. [Loading the Classical Iris Dataset onto A Quantum Circuit](#cml)<br>
A. [ Data Analysis and Description](#data_analysis)<br>
B. [Splitting Data](#data_split)<br>
3. [Training a Quantum Machine Learning Model](#prep)<br>
A. [Data Encoding Using the ZFM](#data_encoding)<br>
B. [Parameterized Quantum Circuit](#ansatz)
4. [Optimise the Problem for Quantum Execution](#optimise)<br>
5. [Execute Using Qiskit Primitives](#execute)<br>
6. [Post-Process Results to Extract Classical Data](#post)<br>
7. [Conclusion](#conclusion)

### 0. Introduction to the QML Practical Session with Qiskit! <a id="welcome"></a>
Welcome to the Quantum Machine Learning (QML) Hands-on Workshop curated for the IBM-CHCP-AIMS Workshop on The Theory of Quantum Learning Algorithms! In this session we demonstate how IBM Research scientists and engineers leverage Qiskit Patterns to execute QML workflows. In particular, we demonstrate an implementation of hybrid quantum classical model in labeling unseen images from the well-known IRIS dataset containing three iris species: Setosa, Versicolour, and Virginica.

QML can be described as a quantum computing paradigm that applies quantum mechanical phenomena such as superposition and entanglement to enable researchers to extract insights and patterns from data.​ At IBM Research, we exploit QML across multiple disciplinaries such healthcare, finance, and chemistry, to augment or complement existing classical workflows.

In this notebook, we implement the steps in building efficient QML workflows by leveraging the high performance of quantum computers through this pipeline we show how by fine tuning parameters in the quantum algorithm using the Qiskit software development kit (SDK) which follows Qiskit Patterns can be useful in mitigating the bottlenecks that arise in quantum hardware.

The Qiskit SDK is a fundamental toolkit for performing quantum experiments. We begin verifying that Qiskit 2.x is installed and setup the required modules for visualising data. Then, we ensure that the IRIS dataset is stored in our local environment.
#### 0.1. Verifying Qiskit installation and checking version <a id="install"></a>

In the below cells, we check the Qiskit version. The minimum version required to run workloads on the new IBM Quantum Platform released in July 2025 is 2.0.0. We then download IRIS dataset using $\texttt{pip}$

In the following section, we implement the QML workflow, in which the classical data  are encoded to quantum states, data encoding is typically the first step in the QML pipeline. The full methodology follows the Qiskit Pattern framework as illustrated in the figure below:
1. Mapping classical inputs to a quantum problem.
2. Optimising the problem for execution on available quantum systems.
3. Executing quantum circuits on simulators and hardware devices using the Qiskit Runtime primitives.
4. Post-processing, returning the result in classical format.
<p style="text-align:center;">
    <img src="images/qiskit_patterns.png" />
</p>
<p style="text-align:center;">
    Figure 0.1. illustrating the Qiskit Pattern framework used in classifying iris species using a QNN. 
</p>
To investigate QML further, we encourage you to visit the  <a href=https://quantum.cloud.ibm.com/learning/en/courses/quantum-machine-learning/introduction> Quantum Machine Learning course material</a> by IBM Quantum. For now, let's verify that we have Qiskit installed on our devices. If you do not have Qiskit on your machine, you can uncomment the cell below to install it, among other modules that we will be using moving forward. 

In [ ]:
# %pip install qiskit qiskit-machine-learning 
# %pip seaborn scikit-learn pandas matplotlib


In [ ]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

#### 0.2. Import Modules <a id="import"></a>


Qiskit is an open-source SDK created by IBM Research to enable quantum researchers and developers to programmatically interface with IBM Quantum hardware and simulators. Over the years, Qiskit has evolved to include powerful modules like Qiskit Addons and Qiskit Functions, which expedite the development of quantum algorithms. Among the Qiskit Functions is the $\texttt{transpiler}$ package, which interprets quantum circuits and optimizes the conversion of abstract qubits to specific qubits on the hardware devices. Because our QML pipeline implements a classical-quantum hybrid stream, we can also leverage common Python modules to optimize certain parts of our workflow. 

To run workflows on IBM Quantum hardware, researchers at IBM Research use Qiskit Runtime, which is the architecture that streamlines computations requiring multiple iterations. Qiskit Runtime enables algorithms to execute significantly faster. 

For the purpose of this demonstration, we will use the Qiskit Runtime Estimator which is a primitive designed to calculate the expectation values of specified observables with respect to quantum states prepared by quantum circuits. For more information, on the packages used, visit the <a href='https://quantum.cloud.ibm.com/docs/en/api/qiskit-ibm-runtime/runtime-service'>documentation on the IBM Quantum Platform</a>.

In [ ]:
import numpy as np
import json
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from qiskit.circuit.library import ZFeatureMap, z_feature_map, RealAmplitudes, EfficientSU2
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BaseEstimatorV2
from qiskit.quantum_info.operators.base_operator import BaseOperator
from scipy.optimize import minimize
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    ALAPScheduleAnalysis,
    ConstrainedReschedule,
    PadDynamicalDecoupling,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import copy
from sklearn.metrics import accuracy_score
from qiskit.primitives import StatevectorEstimator as Estimator  # simulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator2,
    Session,
)
from qiskit_machine_learning.utils import algorithm_globals

<div class="alert alert-block alert-info">
<b>Account Set Up: API Key and CRN</b>

- Enter your API Key and CRN.

**Your Goal:** Please enter your IBM Quantum Platform API Key and CRN in the cell below.

In [ ]:
# ---- TODO : Setup an Instance---
# Enter your own API key and Cloud Resource Name(CRN) from the IBM Quantum Platform


# Read token
with open("tokens.json", "r") as f:
    loaded_data = json.load(f)
api_key = loaded_data['API_key']
crn = loaded_data['CRN_key']

# Construct account object for interacting with Qiskit Runtime service
QiskitRuntimeService.save_account(token=api_key, instance=crn, name="open",overwrite=True, set_as_default=True)


# service = QiskitRuntimeService(name="open")
# service.saved_accounts()

In [ ]:
# Get a backend list - with access to 
service = QiskitRuntimeService()
print("Available backends:")
for backend in service.backends(simulator=False):
    print(backend.name)

avail_backend =service.least_busy(simulator=False)
print(f"\n least busy backend {avail_backend}")

# backend = service.backend("ibm_torino")
backend = avail_backend
print(f"We are using the {backend.name} quantum computer")

#### 0.3. Overview of the QML Pipeline Adapted from Qiskit Patterns<a id="introduction"></a>

In this practical, we will follow an adapted version of the Qiskit Pattern to perform supervised classification by:
1. Loading the classical $\texttt{iris\_dataset}$ and encoding it to quantum information.
2. Generate and adjust the variational/parameterized quantum circuit (PQC) for our quantum neural network (QNN).
3. Use Qiskit Primitives on the trained QNN to predict the iris species.
4. Explore ways to scale the classification problem.

Subroutines involved in each step are included in Figure 0.2., illustrating the full QML pipeline for classifying irises into three groups. For each step, we provide a brief mathematical description supporting our implementations of the Qiskit functions that we will be employing in our classical-quantum classification algorithm.
<p style="text-align:center;">
    <img src="images/qml_workflow_05.png" />
</p>
<p style="text-align:center;">
    Figure 0.2. illustrating the framework for classifying iris species using a QNN. 
</p>

The illustration highlights how current QML pipelines combine classical and quantum information processing methods to perform the classification. The QML pipeline includes a classical-to-quantum conversion step in which classical data is mapped into quantum states, as illustrated in the diagram. 

The encoded data is passed through a trainable variational quantum circuit, also known as an ansatz. This is a parameterized quantum circuit that can be trained to learn patterns in the data. The ansatz typically consists of a series of quantum gates with adjustable parameters, which are optimized during training to minimize a loss function. The goal is to find the optimal parameters that enable the circuit to accurately process the input data and produce the desired output.


Let's begin!

### 1. Loading the Iris Dataset <a id="cml"></a>
Firstly, we load the the Iris dataset from the $\texttt{sklearn.dataset}$ module. We then explore the data using visualization techniques to construct an intuitive picture of the data. 

In [ ]:
# Store the Iris data set from the sklearn module
iris_dataset = datasets.load_iris()

# Print dataset description
print(iris_dataset.DESCR)

We can highlight a few interesting observations from this dataset description:

- There are 150 samples (instances) in the dataset.
- There are four features (attributes) for each class.
- There are three labels (classes) in the dataset.
- The dataset is perfectly balanced, as there are the same number of samples (50) in each class.
- We can see features are not normalized, and their value ranges are different, e.g., $[4.3, 7.9]$ and $[0.1, 2.5]$ for sepal length and petal width, respectively. So, transforming the features to the same scale may be helpful.
- As stated in the table above, feature-to-class correlation in some cases is very high; this may lead us to think that our model should cope well with the dataset.

We only examined the dataset description, but additional properties are available in the `iris_data` object. Now we are going to work with features and labels from the dataset.

##### A. Data Analysis and Description <a id="data_analysis"></a>

For this demonstration, we define the classical $\texttt{iris\_dataset}$ as a set $X$ consisting of $M=4$ data vectors corresponding to each feature, given by, $\begin{align} X & = \{\vec{x}^{(j)} |j \in [M]\} \nonumber \end{align}$ where set $X = 150$ each with four attributes, the sepal length, sepal width, petal length and petal width, respectively.

To improve the accuracy of the classical-to-quantum conversion, the set $M$ of features needs to be prepared for mapping to quantum data such that information is not lost and the performance of the model is not compromised. This requires that we formulate a clear understanding of the classical data so that we can curate optimal quantum circuits in the QNN. 

The code below is used to analyse our data using tables and graphical representations. We also leverage $\texttt{sklearn}$ functions to extract the features and target names for our model. These features are what we will be encoding to a quantum circuit.



In [ ]:
# Feature desciption
features = iris_dataset.data
labels = iris_dataset.target

df = pd.DataFrame(iris_dataset.data, columns=iris_dataset.feature_names)
df["class"] = pd.Series(iris_dataset.target)

sns.pairplot(df, hue="class", palette="tab10")
df

Plots show that feature values range between 1 and 8 cm. To ensure optimal model performance, we use min-max normalization to rescale the feature vectors of the dataset $X$ before encoding. This simple transformation to represent all features on the same scale. Min-max normalisation takes the ratio of the difference between a datum point $x_{k}^{(i)}$ and the minimum entry in the data vector $\vec{x}^{(j)}_k$ to the difference between the maximum and minimum entries over the $M$ data vectors in the dataset $X$. Mathematically, we can write express min-max normalisation as
$\begin{align}x^{'(i)}_{k} & = \frac{x^{(i)}_k - \min{\{x^{(j)}_k|\vec{x}^{(j)}\in [X]\}}}{\max{\{x^{(j)}_k|\vec{x}^{(j)}\in [X]\}} - \min{\{x^{(j)}_k|\vec{x}^{(j)}\in [X]}\}}\nonumber\end{align}$
which produces values that fall between 0 and 1.

We can use `MinMaxScaler` from scikit-learn to perform this. The code in the cell normalizes the feature vectors by mapping the data onto a range $[0, 1]$. When encoding data to quantum phase information, as in the case of our ZFM implementation, we rescale feature vectors as $\vec{x}_{i}^{(j)}\in (0, 2\pi]$.  


In [ ]:
features = MinMaxScaler().fit_transform(features)
df_normalized = pd.DataFrame(features, columns=iris_dataset.feature_names)
df_normalized["class"] = pd.Series(iris_dataset.target)
sns.pairplot(df_normalized, hue="class", palette="tab10")
df_normalized

Plots of the normalized data vectors in $X$ confirm that we have successfully rescaled the classical data values to fall within a unit circle. A close look at the plots shows that shape of the data points is preserved.

##### B. Splitting Data <a id="data_split"></a>



We can now split our normalized dataset into training and testing sets so that the model can be evaluated with unseen data.

In [ ]:
algorithm_globals.random_seed = 123
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.8, random_state=algorithm_globals.random_seed
)

As a golden measure, we train a classical Support Vector Classifier (SVC) using the $\texttt{SVC}$ class from the $\texttt{sklearn.svm}$ module and evaluate the performance of the model. 

In [ ]:
svc = svm.SVC()
_ = svc.fit(train_features, train_labels)  # suppress printing the return value
train_score_c4 = svc.score(train_features, train_labels)
test_score_c4 = svc.score(test_features, test_labels)

print(f"Classical SVC on the training dataset: {train_score_c4:.2f}")
print(f"Classical SVC on the test dataset:     {test_score_c4:.2f}")

The SVC class implements a Support Vector Machine model which is known to perform well on the Iris dataset. Although we compare the results at the end of this pipeline, researchers must be careful to ensure that the problem is suitable for applications of quantum subroutines during assessment of the solution. QML is suitable for more complex datasets which can be represented in Hilbert space without losing information about the structure of the data. 

#### 2. Training a Quantum Machine Learning Model <a id="prep"></a> 

As an example of a quantum model, we'll train a QNN using an Estimator. This involves preparing the quantum circuit, using an Estimator instance to get the observable's expectation value (the network's output), and then using a classical loss function and gradient-based optimization to adjust the parameters until the loss is minimized. 

But before we train a model, let's examine what comprises the QNN model. Two of its central elements are the feature map and ansatz. 

Careful considerations have to be made when constructing a training ansatz in order for it to align to the problem or data. The considerations include:

* Circuit depth: to prevent excessive hardware errors, we must ensure that we do not perform more operations than possible. 
* Parametrized circuit: we must decide on whether parameters of single or entangling gates should be varied.
* Number of Parameters: we must choose the number of parameters such that the overhead of impelementing parametrized quantum gates is reduced.

The a QNN model pipeline is illustrated in Figure 2.1. below.

<p>
    <img src="images/qnn_vqc_illustration.png" />
</p>
<p style="text-align:center;">
    Figure 2.1. An example of a hybrid quantum framework visualizing the quantum circuit, integration of FM with ansatz and optimisation.
</p>


##### A. Data Encoding Using the ZFM <a id="data_encoding"></a>

A data encoding scheme is technique that converts information from one domain to another by converting datapoints that were sampled classically to Hilbert spaces of a quantum computer. There are many such data encoding schemes that can be used to map our classical dataset for quantum processing unit (QPU) execution. The fundamental encoding schemes used in constructing more complex mappings include:
* Basis Encoding
* Amplitude Encoding
* Angle Encoding
* Phase Encoding

For example, dense angle encoding combines angle and phase encoding to facilitate a mapping of two feature values to a single qubit. Similarly, the ZFM extends concepts of phase encoding to include alternating layers of Hadamard and phase gate layers. Given a data vector $\vec{x}$ with $N$ features, the quantum circuit that performs the feature mapping is represented as a unitary operator $U(\vec{x})$ that acts on the initial qubit ground state $\ket{0}^{\otimes N}$, i.e.,
$\begin{align}U(\vec{x})\ket{0}^{\otimes N} & = \ket{\phi(\vec{x})}\nonumber\end{align}$
where $\ket{\phi(\vec{x})}$ is the mapping $\phi$ of data vector $\vec{x}$ consisting of alternating layers of single-qubit gates.

The unitary matrix $U_{ZFM}(\vec{x})$ for the ZFeatureMap (ZFM) is expressed as:

$$U_{ZFM}(\vec{x}) = \left(\bigotimes_{k=1}^{N}P(\vec{x}_k)\right)H^{\otimes N}$$

For the Iris dataset with 4 features, this becomes:

$$U_{ZFM}(\vec{x}) = [P(\vec{x}_1)\otimes P (\vec{x}_2)\otimes P(\vec{x}_3)\otimes P(\vec{x}_4)]H^{\otimes 4}$$

Here:

- $P(\vec{x}_k) = \begin{pmatrix} e^{i \vec{x}_k} & 0 \\ 0 & e^{-i \vec{x}_k} \end{pmatrix}$ is a phase gate applied to each qubit.
- $H^{\otimes N}$ is the Hadamard gate applied to each qubit.

The resulting quantum circuit is applied iteratively for $r$ repetitions to generate a final product state. The code below applies the built-in ZFM Qiskit class which takes in the same number of qubits as the number entries in each feature vector. 




In [ ]:
# encoding the features using an ZFeatureMap
num_features = features.shape[1]
feature_map = ZFeatureMap(feature_dimension=num_features)

# Visualize the circuit
feature_map.decompose().draw(output='mpl', fold=20)


If you look closely at the feature map diagram, you will notice parameters `x[0], ..., x[3]`. These are placeholders for our features.


##### B. Parameterized Quantum Circuit <a id="ansatz"></a> 

In [ ]:
num_qubits = num_features

# Using RealAmplitudes Ansatz for encoding
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
# ansatz = EfficientSU2(num_qubits=num_features, reps=3)

 
# Draw the circuit
ansatz.decompose().draw("mpl")

In the first layer of the QNN ansatz circuit, each  qubit is rotated around the $y$-axis by the parameter $\theta$. This is to say that the single-qubit R<sub>Y</sub> gate is parametrised, the unitary matrix operation given by
$\begin{align}R_Y & = \left(\begin{matrix} \cos(\frac{\theta}{2}) & -\sin(\frac{\theta}{2}) \\ \sin(\frac{\theta}{2}) & \cos(\frac{\theta}{2})\end{matrix}\right)\nonumber\end{align}$
Note that when $\theta = \pi$, applying a Y gate effectively affects the qubit state and introduces a phase shift of $180\degree$. 

In second layer, adjacent qubits are entangled using the $\texttt{CNOT}$ gate. Recall that we can represent the results of a $\texttt{CNOT}$ gate using a Truth Table as shown in table 2.1. below.
<p style="text-align:center;">
    <img src="images/cnot_truth_table.png" />
</p>
<p style="text-align:center;">
    Table 2.1. Truth table for CNOT gate showing the control, target and resultant qubit states.
</p>

Pay attention to the repetitive structure of the ansatz circuit. We define the number of these repetitions using the `reps` parameter.



In [ ]:
# Combine the ZFM with Ansatz
# cr = ClassicalRegister(1, name="cr")
qnn_vqc = QuantumCircuit(QuantumRegister(num_qubits))
qnn_vqc.compose(feature_map, range(num_qubits), inplace=True)
qnn_vqc.compose(ansatz, range(num_qubits), inplace=True)
 
# Display the circuit
qnn_vqc.decompose().draw("mpl", style="clifford", fold=-1)

To interpret the results of the full circuit classically, we require an observable that produces an expectation value. The Qiskit SDK offers the Estimator which can be used to amalgamate quantum states such that we can perform measurements on a subset of the quantum kernel space produced by the QNN. Alternatively, we can use the $\texttt{EstimatorV2}$ to measure some attribute from each qubit by applying an I, X, Y or Z operator to each qubit. In this case, we will estimate the expectation value by computing the number of counts and dividing them by the number of executed shots. This is performed in the code below where where we use a function to run a foward pass.

In [ ]:
# define the QNN function for later
def forward_qnn(
    qnn_qc: QuantumCircuit,
    theta_params: np.ndarray,
    weight_params: np.ndarray,
    estimator: BaseEstimatorV2,
    observable: BaseOperator,
) -> np.ndarray:
    
    num_samples = theta_params.shape[0]
    weights = np.broadcast_to(weight_params, (num_samples, len(weight_params)))
    params = np.concatenate((theta_params, weights), axis=1)
    pub = (qnn_qc, observable, params)
    job = estimator.run([pub])
    result = job.result()[0]
    expectation_values = result.data.evs
 
    return expectation_values


# Define the Z operator observable which produces -1, 0, 1
observable = SparsePauliOp.from_list([("ZX" * (int(num_qubits/2)), 1)])
np.random.seed(42)
weight_params = np.random.rand(len(ansatz.parameters)) * 2 * np.pi
qnn_vqc.decompose().draw('mpl')

In [ ]:
from matplotlib import pyplot as plt
from IPython.display import clear_output

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)


def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

We apply gradient-based loss function which takes in the labels predicted by our model and the groundtruth labels before returning the mean squared errror (MSE). We also apply a function that takes in the PQC parameters as inputs for use by the COBYLA classical optimiser which trains the model by sampling the dynamic weights to decrease the MSE. The result from these functions are used to indicate the accuracy of our QNN. Note that classical deep learning frameworks use a similar technique to in gradient-based techniques. 

In [ ]:
# Global parameters
vqc = qnn_vqc
observables = observable
input_params = train_features
target = train_labels
estimator = Estimator()
gradient = []
iter = 0

def loss(predict: np.ndarray, target: np.ndarray) -> np.ndarray:
    """
     Loss function for calculating the MSE.
    """
    if len(predict.shape) <= 1:
        return ((predict - target) ** 2).mean()
    else:
        raise AssertionError("input should be 1d-array")

def loss_weights(weight_params: np.ndarray) -> np.ndarray:
    """
    Cost function for use by classical optimiser.
    """
    predictions = forward_qnn(
        qnn_qc=vqc,
        theta_params=input_params,
        weight_params=weight_params,
        estimator=estimator,
        observable=observables,
    )
 
    cost = loss(predict=predictions, target=target)
    objective_func_vals.append(cost)
    gradient.append(cost)
 
    global iter
    if iter % 50 == 0:
        print(f"Iter: {iter}, loss: {cost}")
    iter += 1
 
    return cost

### 3. Optimising the Problem for Quantum Execution <a id="optimise"></a>

IBM Quantum systems have different properties such as single and two-qubit gate error rates, read-out rates and qubit coupling maps. In this step of our pipeline, we select the least-busy quantum system for execution. 

In [ ]:
# getting a real backend whith low waiting times
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)


The Qiskit SDK allows us to optimise a logical quantum circuit for QPU execution using a preset pass manager based on the layout of physical qubits on the available quantum system. 

In [ ]:
target = backend.target
# Construct standalone PassManager for handling task scheduling
pass_manager = generate_preset_pass_manager(target=target, optimization_level=3)
pass_manager.scheduling = PassManager(
    [
        # ALAP Schedule Pass class created with target backend
        ALAPScheduleAnalysis(target=target),
        # Rescheduler class for handling time resolution
        ConstrainedReschedule(
            acquire_alignment=target.acquire_alignment,
            pulse_alignment=target.pulse_alignment,
            target=target,
        ),
        # Pass class for dynamic decoupling in idle periods
        PadDynamicalDecoupling(
            target=target,
            dd_sequence=[XGate(), XGate()],
            pulse_alignment=target.pulse_alignment,
        ),
    ]
)
# Execute the defined schedule of passes to transpile your circuit for the device
physical_qubits = pass_manager.run(qnn_vqc)
# Mapp the obervable to the physical layout of the used device
physical_observable = observable.apply_layout(physical_qubits.layout)


### 4. Executing Qiskit Primitives <a id="execute"></a>

We can now execute the optimized circuit on the QPU using Qiskit Primitives. We begin by debugging the circuit using a simulator to loop over the dataset for $b$ batches and $m$ epochs to train our QNN. Notice how decreasing the batch number affects the accuracy of the QNN.

In [ ]:
batch_size = 200
num_epochs = 10
num_samples = len(train_features)
objective_func_vals = []

# Random initial weights for the ansatz. The fixed seed allows to replicate the experiment.
# This was already set above, but you can try to change the seed to see the difference.
# np.random.seed(42)
# weight_params = np.random.rand(len(ansatz.parameters)) * 2 * np.pi

for epoch in range(num_epochs):
    for i in range((num_samples - 1) // batch_size + 1):
        print(f"Epoch: {epoch}, batch: {i}")
        start_i = i * batch_size
        end_i = start_i + batch_size
        train_iris_batch = np.array(train_features[start_i:end_i])
        train_labels_batch = np.array(train_labels[start_i:end_i])
        input_params = train_iris_batch
        target = train_labels_batch
        iter = 0
        res = minimize(
            loss_weights, weight_params, method="COBYLA", options={"maxiter": 100}
        )
        weight_params = res["x"]

### 5. Post-Processing Results to Extract Classical Data <a id="post"></a>

In the final step of our adaptation of the Qiskit Pattern, we determine the training accuracy to interpret the above results.

In [ ]:
pred_train = forward_qnn(vqc, np.array(train_features), res["x"], estimator, observable)
# pred_train = forward(circuit_ibm, np.array(train_images), res['x'], estimator, observable_ibm)
 
print(pred_train)
# Define the ternary thresholds
threshold_low = 0.25
threshold_high = 0.3

# Apply the thresholds to the predictions
pred_train_labels = copy.deepcopy(pred_train)
index = 0
for pred in pred_train:
    if pred < threshold_low:
        pred_train_labels[index] = 0
    elif pred >= threshold_low and pred < threshold_high: 
        pred_train_labels[index] = 2
    else:
        pred_train_labels[index] = 1
    index+=1

print(pred_train_labels)
print(train_labels)
 
accuracy = accuracy_score(train_labels, pred_train_labels)
print(f"Train accuracy: {accuracy * 100}%")

In [ ]:
pred_test = forward_qnn(vqc, np.array(test_features), res["x"], estimator, observable)
# pred_test = forward(circuit_ibm, np.array(test_images), res['x'], estimator, observable_ibm)
 
print(pred_test)
 
# Apply the thresholds to the predictions
pred_test_labels = copy.deepcopy(pred_test)
index = 0
for pred in pred_test:
    if pred < threshold_low:
        pred_test_labels[index] = 0
    elif pred >= threshold_low and pred < threshold_high: 
        pred_test_labels[index] = 2
    else:
        pred_test_labels[index] = 1
    index+=1

print(pred_test_labels)
print(test_labels)
 
accuracy = accuracy_score(test_labels, pred_test_labels)
print(f"Test accuracy: {accuracy * 100}%")

When we use 200 batches, the accuracy of the model exhibits moderate performance, achieving a model training accuracy of 67.5%. This prompts us to investigate potential issues, adjust the ansatz, and optimize our model. To guide our investigation, we can begin by identifying the following plausible causes:
* We may have insufficient training iterations, implying that the process may not have been run for enough epochs.
* The circuit architecture of the ansatz may not be optimal, leading to poor entanglement and limited classification.

This means that to improve the performance of the model, we can increase the number of training epochs to ensure sufficient iterations for the optimizer to find the best parameters. We can also use a different encoding scheme and optimize the ansatz to improve entanglement to avoid overfitting. The code below is used to check for convergence in the optimization which can help us verify whether we constructed an ansatz with poor learnability.

In [ ]:
obj_func_vals_first = objective_func_vals
# import matplotlib.pyplot as plt
 
plt.figure(figsize=(12, 6))
plt.plot(obj_func_vals_first, label="Iris Dataset Ansatz")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()

We can check the number of features that were not classified correctly to understand how we can improve the QNN.

In [ ]:
missed = []
for i in range(len(test_labels)):
    if [i] != test_labels[i]:
        missed.append(test_labels[i])
print(len(missed))

### 6. Conclusion <a id="conclusion"></a>

This notebook explored the potential of Quantum Machine Learning for classifying the Iris dataset. While quantum models achieved promising results, they still lag behind their classical counterparts in terms of accuracy and resource efficiency. However, the potential for future advancements in quantum hardware and algorithms suggests that quantum ML will eventually reach parity with classical ML. Quantum models achieved a training accuracy of 67.5% on the Iris dataset using four features, surpassing classical models trained on the same features. Reducing the number of features negatively impacted the performance of both classical and quantum models. In conclusion, the results suggest uantum models require further optimization for better entanglement and learnability.